In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-02-23 23:25:13,000 INFO: Initializing external client
2025-02-23 23:25:13,000 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-23 23:25:13,718 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214632
Fetching data from 2025-01-26 04:25:12.999370+00:00 to 2025-02-24 03:25:12.999370+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.91s) 


In [4]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-02-23 05:00:00
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,3,2025-02-23 05:00:00
2,0,1,1,0,2,2,6,5,5,3,...,17,26,56,84,117,119,66,18,4,2025-02-23 05:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-02-23 05:00:00
4,1,1,2,5,6,3,2,5,2,2,...,4,1,2,2,1,2,4,13,7,2025-02-23 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,1,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,259,2025-02-23 05:00:00
247,0,3,1,1,3,0,2,0,0,3,...,0,0,2,0,0,1,2,4,260,2025-02-23 05:00:00
248,4,0,3,5,11,17,19,26,34,30,...,16,20,25,24,11,6,1,4,261,2025-02-23 05:00:00
249,7,12,33,53,62,103,119,110,144,100,...,32,33,37,12,8,4,8,4,262,2025-02-23 05:00:00


In [5]:
predictions = fetch_next_hour_predictions()

2025-02-23 23:25:23,929 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-23 23:25:23,934 INFO: Initializing external client
2025-02-23 23:25:23,935 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-02-23 23:25:24,376 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214632
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.39s) 
Current UTC time: 2025-02-24 04:25:23.929552+00:00
Next hour: 2025-02-24 05:00:00+00:00
Found 251 records


In [6]:
predictions


,pickup_location_id,predicted_demand,pickup_hour
0,100,69.0,2025-02-24 05:00:00+00:00
1,78,0.0,2025-02-24 05:00:00+00:00
2,180,0.0,2025-02-24 05:00:00+00:00
3,212,0.0,2025-02-24 05:00:00+00:00
4,24,14.0,2025-02-24 05:00:00+00:00
...,...,...,...
246,87,26.0,2025-02-24 05:00:00+00:00
247,21,0.0,2025-02-24 05:00:00+00:00
248,14,1.0,2025-02-24 05:00:00+00:00
249,123,0.0,2025-02-24 05:00:00+00:00


In [7]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [8]:
top10

[237, 79, 236, 162, 164, 138, 239, 68, 163, 234]

In [9]:
top10

[237, 79, 236, 162, 164, 138, 239, 68, 163, 234]

In [10]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
246    False
247    False
248    False
249    False
250    False
Name: pickup_location_id, Length: 251, dtype: bool

In [11]:
features_cp = features.copy()

In [12]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

In [13]:
historical_df

,datetime,rides
0,2025-01-26 05:00:00,7.0
1,2025-01-26 06:00:00,14.0
2,2025-01-26 07:00:00,23.0
3,2025-01-26 08:00:00,60.0
4,2025-01-26 09:00:00,123.0
...,...,...
668,2025-02-23 01:00:00,72.0
669,2025-02-23 02:00:00,23.0
670,2025-02-23 03:00:00,3.0
671,2025-02-23 04:00:00,10.0
